In [2]:
from cassandra.cluster import Cluster, PlainTextAuthProvider
from cassandra.cluster import (
    NoHostAvailable,
    OperationTimedOut
)
import os, sys
import pandas as pd, numpy as np

CASS_HOST = os.getenv('CASS_HOST') or 'localhost'
CASS_USER = os.getenv('CASS_USER') or 'cassandra'
CASS_PASS = os.getenv('CASS_PASS') or 'changeme'

In [3]:
weps = pd.read_csv('../../data/weapons.csv')
arm = pd.read_csv('../../data/armor.csv')

In [8]:
weps.columns

Index(['Name', 'Cost', 'Damage', 'Weight', 'Properties', 'type', 'cost',
       'cost_unit', 'dmg_die', 'dmg_type', 'dmg_die_qty', 'dmg_die_size',
       'dmg', 'props', 'normal_range', 'long_range', 'mod', '2hnd_die_qty',
       '2hnd_die_size', '2hnd_dmg', 'weight', 'wgt'],
      dtype='object')

In [9]:
arm.columns

Index(['Name', 'minSTR', 'stealth_disadv', 'Weight', 'type', 'baseAC', 'modAC',
       'cost_amt', 'cost_unit', 'Don', 'Doff'],
      dtype='object')

In [4]:
auth_provider = PlainTextAuthProvider(
    username = CASS_USER,
    password = CASS_PASS
)
print(CASS_HOST,CASS_USER,CASS_PASS)
cluster = Cluster(
    [CASS_HOST],
    auth_provider=auth_provider
)
session = cluster.connect()

localhost cassandra changeme


In [23]:
session.execute('use dndio;')
for i, row in arm.iterrows():
    # print(row)
    query = """ 
INSERT INTO armor (
    id,
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    uuid(),
    {},
    {},
    '{}',
    '{}',
    '{}',
    {},
    '{}',
    '{}',
    {},
    '{}',
    {}
);
""".format(
    row['baseAC'],row['cost_amt'],row['cost_unit'],row['Doff'],row['Don'],
    row['minSTR'],row['modAC'],row['Name'],str(row['stealth_disadv']).lower(),row['type'],row['Weight']
)
    print(query)
    session.execute(query)
    # print(query)
    # ['Name', 'minSTR', 'stealth_disadv', 'Weight', 'type', 'baseAC', 'modAC',
    #    'cost_amt', 'cost_unit', 'Don', 'Doff']

 
INSERT INTO armor (
    id,
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    uuid(),
    11,
    5,
    'gp',
    '1 minute',
    '1 minute',
    0,
    'DEX',
    'Padded',
    true,
    'light',
    8
);

 
INSERT INTO armor (
    id,
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    uuid(),
    11,
    10,
    'gp',
    '1 minute',
    '1 minute',
    0,
    'DEX',
    'Leather',
    false,
    'light',
    10
);

 
INSERT INTO armor (
    id,
    baseac,
    cost_amt,
    cost_unit,
    doff,
    don,
    minstr,
    modac,
    name,
    stealth_disadv,
    type,
    weight
) VALUES (
    uuid(),
    12,
    45,
    'gp',
    '1 minute',
    '1 minute',
    0,
    'DEX',
    'Studded Leather',
    false,
    'light',
    13
);

 
INSERT INTO armor (
    id,
    baseac,
    cost_amt,
 

In [ ]:
weps.columns
#columns in db needed...name text, cost_unit text, cost_amt smallint, properties(list), type text, norm_range smallint, long_range smallint, mod (list<text>)
#dmg list<frozen<map<text,frozen<map<smallint,smallint>>>>>>, 
#dmg2hnd list<frozen<map<text,frozen<map<smallint,smallint>>>>>>, 
#weight float?
# 

Index(['Name', 'Cost', 'Damage', 'Weight', 'Properties', 'type', 'cost',
       'cost_unit', 'dmg_die', 'dmg_type', 'dmg_die_qty', 'dmg_die_size',
       'dmg', 'props', 'normal_range', 'long_range', 'mod', '2hnd_die_qty',
       '2hnd_die_size', '2hnd_dmg', 'weight', 'wgt'],
      dtype='object')

In [82]:
weps.drop(columns=[
    'Cost','Damage','Weight','Properties','dmg_die','dmg_type','dmg_die_qty','dmg_die_size','2hnd_die_qty','2hnd_die_size','weight'
])


,Name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,2hnd_dmg,wgt
0,Club,Melee,1,sp,{'bludgeoning': {'1': '4'}},['Light'],5,5,['STR'],{'bludgeoning': {'1': '4'}},2.00
1,Dagger,Melee,2,gp,{'piercing': {'1': '4'}},"['Finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",{'piercing': {'1': '4'}},1.00
2,Greatclub,Melee,2,sp,{'bludgeoning': {'1': '8'}},['Two-handed'],5,5,['STR'],{'bludgeoning': {'1': '8'}},10.00
3,Handaxe,Melee,5,gp,{'slashing': {'1': '6'}},"['Light', 'thrown (20/60)']",20,60,['STR'],{'slashing': {'1': '6'}},2.00
4,Javelin,Melee,5,sp,{'piercing': {'1': '6'}},['Thrown (30/120)'],5,5,['STR'],{'piercing': {'1': '6'}},2.00
5,Light hammer,Melee,2,gp,{'bludgeoning': {'1': '4'}},"['Light', 'thrown (20/60)']",20,60,['STR'],{'bludgeoning': {'1': '4'}},2.00
6,Mace,Melee,5,gp,{'bludgeoning': {'1': '6'}},[],5,5,['STR'],{'bludgeoning': {'1': '6'}},4.00
7,Quarterstaff,Melee,2,sp,{'bludgeoning': {'1': '6'}},['Versatile (1d8)'],5,5,['STR'],{'bludgeoning': {'1': '8'}},4.00
8,Sickle,Melee,1,gp,{'slashing': {'1': '4'}},['Light'],5,5,['STR'],{'slashing': {'1': '4'}},2.00
9,Spear,Melee,1,gp,{'piercing': {'1': '6'}},"['Thrown (20/60)', 'versatile (1d8)']",5,5,['STR'],{'piercing': {'1': '8'}},3.00


create table weapons (
    id uuid,
    name text,
    type text,
    cost_amt smallint,
    cost_unit text,
    dmg list<frozen<map<text,frozen<map<smallint,smallint>>>>>,
    dmg2hnd list<frozen<map<text,frozen<map<smallint,smallint>>>>>,
    mod list<text>,
    norm_range smallint,
    long_range smallint,
    props list<text>,
    weight double,
    PRIMARY KEY (id)
);

In [83]:
weps['props']
weps['props'] = weps.apply(
    lambda row: ''.join([ row['props'][i].lower() for i in range(len(row['props']))]),
    axis=1
)
display(weps)


,Name,Cost,Damage,Weight,Properties,type,cost,cost_unit,dmg_die,dmg_type,...,dmg,props,normal_range,long_range,mod,2hnd_die_qty,2hnd_die_size,2hnd_dmg,weight,wgt
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Melee,1,sp,1d4,bludgeoning,...,{'bludgeoning': {'1': '4'}},['light'],5,5,['STR'],1,4.0,{'bludgeoning': {'1': '4'}},2 lb.,2.00
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Melee,2,gp,1d4,piercing,...,{'piercing': {'1': '4'}},"['finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",1,4.0,{'piercing': {'1': '4'}},1 lb.,1.00
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Melee,2,sp,1d8,bludgeoning,...,{'bludgeoning': {'1': '8'}},['two-handed'],5,5,['STR'],1,8.0,{'bludgeoning': {'1': '8'}},10 lb.,10.00
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Melee,5,gp,1d6,slashing,...,{'slashing': {'1': '6'}},"['light', 'thrown (20/60)']",20,60,['STR'],1,6.0,{'slashing': {'1': '6'}},2 lb.,2.00
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Melee,5,sp,1d6,piercing,...,{'piercing': {'1': '6'}},['thrown (30/120)'],5,5,['STR'],1,6.0,{'piercing': {'1': '6'}},2 lb.,2.00
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Melee,2,gp,1d4,bludgeoning,...,{'bludgeoning': {'1': '4'}},"['light', 'thrown (20/60)']",20,60,['STR'],1,4.0,{'bludgeoning': {'1': '4'}},2 lb.,2.00
6,Mace,5 gp,1d6 bludgeoning,4 lb.,—,Melee,5,gp,1d6,bludgeoning,...,{'bludgeoning': {'1': '6'}},[],5,5,['STR'],1,6.0,{'bludgeoning': {'1': '6'}},4 lb.,4.00
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Melee,2,sp,1d6,bludgeoning,...,{'bludgeoning': {'1': '6'}},['versatile (1d8)'],5,5,['STR'],1,8.0,{'bludgeoning': {'1': '8'}},4 lb.,4.00
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Melee,1,gp,1d4,slashing,...,{'slashing': {'1': '4'}},['light'],5,5,['STR'],1,4.0,{'slashing': {'1': '4'}},2 lb.,2.00
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Melee,1,gp,1d6,piercing,...,{'piercing': {'1': '6'}},"['thrown (20/60)', 'versatile (1d8)']",5,5,['STR'],1,8.0,{'piercing': {'1': '8'}},3 lb.,3.00


In [84]:
weps.drop(columns=[
    'Cost','Damage','Weight','Properties','dmg_die','dmg_type','dmg_die_qty','dmg_die_size','2hnd_die_qty','2hnd_die_size','weight'
],inplace=True)
weps


,Name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,2hnd_dmg,wgt
0,Club,Melee,1,sp,{'bludgeoning': {'1': '4'}},['light'],5,5,['STR'],{'bludgeoning': {'1': '4'}},2.00
1,Dagger,Melee,2,gp,{'piercing': {'1': '4'}},"['finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",{'piercing': {'1': '4'}},1.00
2,Greatclub,Melee,2,sp,{'bludgeoning': {'1': '8'}},['two-handed'],5,5,['STR'],{'bludgeoning': {'1': '8'}},10.00
3,Handaxe,Melee,5,gp,{'slashing': {'1': '6'}},"['light', 'thrown (20/60)']",20,60,['STR'],{'slashing': {'1': '6'}},2.00
4,Javelin,Melee,5,sp,{'piercing': {'1': '6'}},['thrown (30/120)'],5,5,['STR'],{'piercing': {'1': '6'}},2.00
5,Light hammer,Melee,2,gp,{'bludgeoning': {'1': '4'}},"['light', 'thrown (20/60)']",20,60,['STR'],{'bludgeoning': {'1': '4'}},2.00
6,Mace,Melee,5,gp,{'bludgeoning': {'1': '6'}},[],5,5,['STR'],{'bludgeoning': {'1': '6'}},4.00
7,Quarterstaff,Melee,2,sp,{'bludgeoning': {'1': '6'}},['versatile (1d8)'],5,5,['STR'],{'bludgeoning': {'1': '8'}},4.00
8,Sickle,Melee,1,gp,{'slashing': {'1': '4'}},['light'],5,5,['STR'],{'slashing': {'1': '4'}},2.00
9,Spear,Melee,1,gp,{'piercing': {'1': '6'}},"['thrown (20/60)', 'versatile (1d8)']",5,5,['STR'],{'piercing': {'1': '8'}},3.00


In [85]:
import json
for i,row in weps.iterrows():
    try:
        d1 = row['dmg']
        d1=d1.replace('\'','"')
        
        d1 = json.loads(d1)
        for k,v in d1[list(d1.keys())[0]].items():
            d1[list(d1.keys())[0]] = {int(k):int(v)}
        print(d1)
    except Exception as e:
        d1 = {}
        print(e)
    print(d1)
    weps.at[i,'dmg'] = d1.copy()

    try:
        d1 = row['2hnd_dmg']
        d1=d1.replace('\'','"')
        d1 = json.loads(d1)
        for k,v in d1[list(d1.keys())[0]].items():
            d1[list(d1.keys())[0]] = {int(k):int(v)}
        print(d1)
    except Exception as e:
        d1 = {}
        print(e)
    weps.at[i,'2hnd_dmg'] = d1.copy()
    print(d1)
weps

{'bludgeoning': {1: 4}}
{'bludgeoning': {1: 4}}
{'bludgeoning': {1: 4}}
{'bludgeoning': {1: 4}}
{'piercing': {1: 4}}
{'piercing': {1: 4}}
{'piercing': {1: 4}}
{'piercing': {1: 4}}
{'bludgeoning': {1: 8}}
{'bludgeoning': {1: 8}}
{'bludgeoning': {1: 8}}
{'bludgeoning': {1: 8}}
{'slashing': {1: 6}}
{'slashing': {1: 6}}
{'slashing': {1: 6}}
{'slashing': {1: 6}}
{'piercing': {1: 6}}
{'piercing': {1: 6}}
{'piercing': {1: 6}}
{'piercing': {1: 6}}
{'bludgeoning': {1: 4}}
{'bludgeoning': {1: 4}}
{'bludgeoning': {1: 4}}
{'bludgeoning': {1: 4}}
{'bludgeoning': {1: 6}}
{'bludgeoning': {1: 6}}
{'bludgeoning': {1: 6}}
{'bludgeoning': {1: 6}}
{'bludgeoning': {1: 6}}
{'bludgeoning': {1: 6}}
{'bludgeoning': {1: 8}}
{'bludgeoning': {1: 8}}
{'slashing': {1: 4}}
{'slashing': {1: 4}}
{'slashing': {1: 4}}
{'slashing': {1: 4}}
{'piercing': {1: 6}}
{'piercing': {1: 6}}
{'piercing': {1: 8}}
{'piercing': {1: 8}}
{'piercing': {1: 8}}
{'piercing': {1: 8}}
{'piercing': {1: 8}}
{'piercing': {1: 8}}
{'piercing': {1:

,Name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,2hnd_dmg,wgt
0,Club,Melee,1,sp,{'bludgeoning': {1: 4}},['light'],5,5,['STR'],{'bludgeoning': {1: 4}},2.00
1,Dagger,Melee,2,gp,{'piercing': {1: 4}},"['finesse', 'light', 'thrown (20/60)']",20,60,"['STR', 'DEX']",{'piercing': {1: 4}},1.00
2,Greatclub,Melee,2,sp,{'bludgeoning': {1: 8}},['two-handed'],5,5,['STR'],{'bludgeoning': {1: 8}},10.00
3,Handaxe,Melee,5,gp,{'slashing': {1: 6}},"['light', 'thrown (20/60)']",20,60,['STR'],{'slashing': {1: 6}},2.00
4,Javelin,Melee,5,sp,{'piercing': {1: 6}},['thrown (30/120)'],5,5,['STR'],{'piercing': {1: 6}},2.00
5,Light hammer,Melee,2,gp,{'bludgeoning': {1: 4}},"['light', 'thrown (20/60)']",20,60,['STR'],{'bludgeoning': {1: 4}},2.00
6,Mace,Melee,5,gp,{'bludgeoning': {1: 6}},[],5,5,['STR'],{'bludgeoning': {1: 6}},4.00
7,Quarterstaff,Melee,2,sp,{'bludgeoning': {1: 6}},['versatile (1d8)'],5,5,['STR'],{'bludgeoning': {1: 8}},4.00
8,Sickle,Melee,1,gp,{'slashing': {1: 4}},['light'],5,5,['STR'],{'slashing': {1: 4}},2.00
9,Spear,Melee,1,gp,{'piercing': {1: 6}},"['thrown (20/60)', 'versatile (1d8)']",5,5,['STR'],{'piercing': {1: 8}},3.00


In [50]:
weps.columns

Index(['Name', 'type', 'cost', 'cost_unit', 'dmg', 'props', 'normal_range',
       'long_range', 'mod', '2hnd_dmg', 'wgt'],
      dtype='object')

In [86]:
session.execute('use dndio;')
for i, row in weps.iterrows():
    # print(row)
    query = """ 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    '{}',
    '{}',
    {},
    '{}',
    [{}],
    [{}],
    {},
    {},
    {},
    {},
    {}
);
""".format(
    row['Name'],row['type'],row['cost'],row['cost_unit'],
    row['dmg'],row['2hnd_dmg'],row['mod'],row['normal_range'],
    row['long_range'],row['props'],row['wgt']
)
    print(query)
    session.execute(query)
    # ['Name', 'type', 'cost', 'cost_unit', 'dmg', 'props', 'normal_range',
    #    'long_range', 'mod', '2hnd_dmg', 'wgt'],

 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    'Club',
    'Melee',
    1,
    'sp',
    [{'bludgeoning': {1: 4}}],
    [{'bludgeoning': {1: 4}}],
    ['STR'],
    5,
    5,
    ['light'],
    2.0
);

 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    'Dagger',
    'Melee',
    2,
    'gp',
    [{'piercing': {1: 4}}],
    [{'piercing': {1: 4}}],
    ['STR', 'DEX'],
    20,
    60,
    ['finesse', 'light', 'thrown (20/60)'],
    1.0
);

 
INSERT INTO weapons (
    id,
    name,
    type,
    cost_amt,
    cost_unit,
    dmg,
    dmg2hnd,
    mod,
    norm_range,
    long_range,
    props,
    weight
) VALUES (
    uuid(),
    'Greatclub',
    'Melee',
    2,
    'sp',
    [{'bludgeoning': {1: 8}}],
    [

In [97]:
class_levels = pd.read_csv('../../data/class_by_level.csv')
class_levels

,level,prof_bonus,features,class,class_specific,spellslots
0,1,2,"['Rage', ' Unarmored Defense']",Barbarian,"{'rages': 2, 'rage_dmg': 2}",NaN
1,2,2,"['Reckless Attack', ' Danger Sense']",Barbarian,"{'rages': 2, 'rage_dmg': 2}",NaN
2,3,2,"['Primal Path', ' Primal Knowledge (Optional)']",Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
3,4,2,['Ability Score Improvement'],Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
4,5,3,"['Extra Attack', ' Fast Movement']",Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
...,...,...,...,...,...,...
255,16,5,['Ability Score Improvement'],Artificer,"{'infusions_known': 10, 'infused_items': 5, 'c...","{1: 4, 2: 3, 3: 3, 4: 2, 5: 0}"
256,17,6,['-'],Artificer,"{'infusions_known': 10, 'infused_items': 5, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 1}"
257,18,6,['Magic Item Master'],Artificer,"{'infusions_known': 12, 'infused_items': 6, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 1}"
258,19,6,['Ability Score Improvement'],Artificer,"{'infusions_known': 12, 'infused_items': 6, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 2}"


In [ ]:
for i, row in class_levels.iterrows():
    if isinstance(row['spellslots'],float):
        sp = {}
    else:
        sp = row['spellslots']
    if isinstance(row['class_specific'],float): # is None:
        cs = {}
    else:
        cs = row['class_specific']
    if isinstance(row['features'],float):
        feat = []
    else:
        feat = row['features']
        # for f in row['features']:

        #     if '\'' in f
        #         f
    query = """ 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    '{}-{}',
    {},
    {},
    {},
    '{}',
    {},
    {}
)
""".format(
    row['class'],row['level'],row['level'],row['prof_bonus'],feat,row['class'],cs,sp
)
    try:
        session.execute(query)
    except:
        print("couldnt execute: \n{}".format(query))

couldnt execute: 
 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    'Ranger-8',
    8,
    3,
    ['Ability Score Improvement', 'Land\'s Stride', '  Martial Versatility (Optional)'],
    'Ranger',
    {'spells_known': 5},
    {1: 4, 2: 3, 3: 0, 4: 0, 5: 0}
)

couldnt execute: 
 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    'Ranger-10',
    10,
    4,
    ['Natural Explorer Improvement', ' Hide in Plain Sight', '  Deft Explorer Feature (Optional)  ', 'Nature\'s Veil (Optional)'],
    'Ranger',
    {'spells_known': 6},
    {1: 4, 2: 3, 3: 2, 4: 0, 5: 0}
)

couldnt execute: 
 
INSERT INTO classes (
    class_id,
    level,
    prof_bonus,
    features,
    class,
    class_specific,
    spellslots
) VALUES (
    'Rogue-1',
    1,
    2,
    ['Expertise', ' Sneak Attack', 'Thieves\' Cant'],
    'Rogue',
    {'

In [5]:
spells = pd.read_csv('../../data/parsed_spells.csv')


In [6]:
spells.head(10)

,Lvl,Name,Save,Dmg,DmgType,HigherLvlIncrease,Upcast,dmgCpy,hard_mod,mult,upcast,dmg,addspellmod,time,range,components,duration,classes,conc
0,0,Acid Splash,DEX,1d6,Acid,True,NaN,1d6,0,1,NaN,"{1: {'Acid': [{1: 6}]}, 5: {'Acid': [{2: 6}]},...",0,1 action,60 feet,"V, S",Instantaneous,"['Artificer', 'Sorcerer', 'Wizard']",False
1,0,Booming Blade,NaN,1d8,Thunder,True,NaN,1d8,0,1,NaN,{1: {'Thunder': [{1: 8}]}},0,1 action,Self (5-foot radius),"S, M (a melee weapon worth at least 1 sp)",1 round,"['Artificer', 'Sorcerer', 'Warlock', 'Wizard']",False
2,0,Chill Touch,NaN,1d8,Necrotic,True,NaN,1d8,0,1,NaN,"{1: {'Necrotic': [{1: 8}]}, 5: {'Necrotic': [{...",0,1 action,120 feet,"V, S",1 round,"['Sorcerer', 'Warlock', 'Wizard']",False
3,0,Create Bonfire,DEX,1d8,Fire,True,NaN,1d8,0,1,NaN,"{1: {'Fire': [{1: 8}]}, 5: {'Fire': [{2: 8}]},...",0,1 action,60 feet,"V, S","Concentration, up to 1 minute","['Artificer', 'Druid', 'Sorcerer', 'Warlock', ...",True
4,0,Eldritch Blast,NaN,1d10,Force,True,NaN,1d10,0,1,NaN,{1: {'Force': [{1: 10}]}},0,1 action,120 feet,"V, S",Instantaneous,['Warlock'],False
5,0,Fire Bolt,NaN,1d10,Fire,True,NaN,1d10,0,1,NaN,{1: {'Fire': [{1: 10}]}},0,1 action,120 feet,"V, S",Instantaneous,"['Artificer', 'Sorcerer', 'Wizard']",False
6,0,Frostbite,CON,1d6,Cold,True,NaN,1d6,0,1,NaN,"{1: {'Cold': [{1: 6}]}, 5: {'Cold': [{2: 6}]},...",0,1 action,60 feet,"V, S",Instantaneous,"['Druid', 'Sorcerer', 'Warlock', 'Wizard', 'Ar...",False
7,0,Green-Flame Blade,NaN,1d8,Fire,True,NaN,1d8,0,1,NaN,{1: {'Fire': [{1: 8}]}},0,1 action,Self (5-foot radius),"S, M (a melee weapon worth at least 1 sp)",Instantaneous,"['Artificer', 'Sorcerer', 'Warlock', 'Wizard']",False
8,0,Infestation,CON,1d6,Poison,True,NaN,1d6,0,1,NaN,"{1: {'Poison': [{1: 6}]}, 5: {'Poison': [{2: 6...",0,1 action,30 feet,"V, S, M (a living flea)",Instantaneous,"['Druid', 'Sorcerer', 'Warlock', 'Wizard']",False
9,0,Lightning Lure,STR,1d8,Lightning,True,NaN,1d8,0,1,NaN,"{1: {'Lightning': [{1: 8}]}, 5: {'Lightning': ...",0,1 action,Self (15-foot radius),V,Instantaneous,"['Artificer', 'Sorcerer', 'Warlock', 'Wizard']",False


In [14]:
spells.columns

Index(['Lvl', 'Name', 'Save', 'Dmg', 'DmgType', 'HigherLvlIncrease', 'Upcast',
       'dmgCpy', 'hard_mod', 'mult', 'upcast', 'dmg', 'addspellmod', 'time',
       'range', 'components', 'duration', 'classes', 'conc'],
      dtype='object')

In [15]:
spells['Lvl']

0      0
1      0
2      0
3      0
4      0
      ..
171    9
172    9
173    9
174    9
175    9
Name: Lvl, Length: 176, dtype: int64

In [ ]:
session.execute('use dndio;')
for i, row in spells.iterrows():
    if row['upcast'] is None:
        uc = {}
    if row['dmg'] is None:
        dmg = {}
    query = """  
INSERT INTO spells (
id,
name,
save,
hard_mod,
mult,
addspellmod,
upcast,
dmg,
time,
range,
components,
duration,
classes,
conc,
lvl
) VALUES (
    uuid(),
    '{}',
    '{}',
    {},
    {},
    {},
    {},
    {},
    '{}',
    '{}',
    '{}',
    '{}',
    {},
    {},
    {}
);
""".format(row['Name'],row['Save'],row['hard_mod'],row['mult'],bool(row['addspellmod']),uc,dmg,row['time'].strip(),row['range'].strip(),row['components'].strip(),row['duration'],row['classes'],row['conc'],row['Lvl'])
    print(query)
    session.execute(query)

  
INSERT INTO spells (
id,
name,
save,
hard_mod,
mult,
addspellmod,
upcast,
dmg,
time,
range,
components,
duration,
classes,
conc,
lvl
) VALUES (
    uuid(),
    'Acid Splash',
    'DEX',
    0,
    1,
    True,
    nan,
    {1: {'Acid': [{1: 6}]}, 5: {'Acid': [{2: 6}]}, 11: {'Acid': [{3: 6}]}, 17: {'Acid': [{4: 6}]}},
    '1 action',
    '60 feet',
    'V, S',
    'Instantaneous',
    ['Artificer', 'Sorcerer', 'Wizard'],
    False,
    0
);



InvalidRequest: Error from server: code=2200 [Invalid query] message="Invalid FLOAT constant (NaN) for "upcast" of type frozen<map<smallint, frozen<map<smallint, smallint>>>>"